<a href="https://colab.research.google.com/github/LordLean/Extracting-Green-Bonds-Use-of-Proceeds/blob/main/QA_Pipeline_barebones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retieval

## Answer Retriever


In [2]:
!pip install rank-bm25

!pip install PyPDF2

!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 217 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.3 MB/s 


In [3]:
import numpy as np

import tabula
from rank_bm25 import BM25Okapi
from PyPDF2 import PdfReader

In [4]:
class TableReader:

  def __init__(self, pdf):
    self.pdf = pdf
    self.dfs = None

  def read_pages(self, pages="all", multiple_tables=True, stream=True):
    '''
    Return tables discovered within pdf.
    '''
    self.dfs = tabula.read_pdf(self.pdf, pages=pages, multiple_tables=multiple_tables, stream=stream)
    self.__clean_dfs()
    return self.dfs

  def __clean_dfs(self, thresh=2):
    self.dfs = [df.dropna(thresh=thresh) for df in self.dfs]


class Reader:

  def __init__(self, filename):
    self.reader = PdfReader(filename)
    self.tb = TableReader(filename)
    self.page_viewer = {page_num : {} for page_num in range(self.reader.numPages)}
    self.idx2page_item = []
  
  def __extract_text(self,):
    '''
    Page-wise text extraction and tokenize for BM25.
    '''
    text_index_mem = 0
    # List to store each tokenized corpus
    tokenized_corpus_list = []
    for i in range(self.reader.numPages):
      raw_text = self.reader.getPage(i).extractText()
      self.page_viewer[i]["raw_text"] = raw_text
      # Split text
      corpus = raw_text.split("\n \n")
      # Store results.
      self.page_viewer[i]["corpus"] = corpus
      for item in corpus:
        self.idx2page_item.append((i, item)) # page,textItem
      # Tokenize
      tokenized_corpus = [doc.split(" ") for doc in corpus]
      tokenized_corpus_list.append(tokenized_corpus)
    # BM25 computations only after the complete tokenized corpus is collated. 
    # Merge tokenized corpus'.
    tokenized_corpus_complete = [item for sublist in tokenized_corpus_list for item in sublist]
    # BM25
    self.bm25 = BM25Okapi(tokenized_corpus_complete)

  def __extract_tables(self):
    '''
    Page-wise table extractor.
    '''
    for i in range(self.reader.numPages):
      # page=0 will throw error using tabula.
      page = str(i+1)
      self.page_viewer[i]["tables"] = self.tb.read_pages(pages=page)

  def extract_pdf(self):
    # Extract data
    self.__extract_text()
    # self.__extract_tables()

  def print_page(self, page_num):
    '''
    Print separated sections of text given a page.
    '''
    corpus = self.page_viewer[page_num]["corpus"]
    for item in (corpus):
      print("\n{}\n".format("-"*60))
      print(item)
    print("\n{}\n".format("-"*60))
    for df in self.page_viewer[page_num]["tables"]:
      print(df.style)
      display(df)

  def __score(self, queries, weights):
    '''
    Compute the average BM25 score of each given query on each page of text.
    '''
    self.ranked_scores = []
    for query in queries:
      # tokenize query by whitespace.
      tokenized_query = query.split()
      # Compute score.
      doc_scores = self.bm25.get_scores(tokenized_query)
      self.ranked_scores.append(doc_scores)
    # Compute average (weighted) score against all queries.
    if not len(weights):
      # Equal weighting.
      self.average_score = np.average(self.ranked_scores, axis=0)
    elif len(queries) != len(weights):
        # Unequal number of elements.
        raise ValueError("Number of query and weight elements passed must be equal.")
    else:
      # Weighted average.
      self.average_score = np.average(self.ranked_scores, weights=weights, axis=0)
 
  def get_ranked_texts(self, queries, weights=[], n=5):
    '''
    Return n pages which scored highest using BM25.
    '''
    # Run score method to calculate BM25.
    self.__score(queries, weights)
    try:
      idx = sorted(range(len(self.average_score)), key=lambda i: self.average_score[i], reverse=True)[:n]
    except IndexError:
      idx = sorted(range(len(self.average_score)), key=lambda i: self.average_score[i], reverse=True)
    final_results = []
    for i in range(n):
      page_num, text = self.idx2page_item[idx[i]]
      # tables = self.page_viewer[page_num]["tables"]
      # final_results.append({"page_num":page_num, "text":text, "tables":tables})
      final_results.append(text)

    return final_results
    

## Answer Re-ranker (Neural: BERT / T5)

In [5]:
!pip install pygaggle

!pip install transformers==4.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 7.3 MB 11.2 MB/s 
     |████████████████████████████████| 63.3 MB 17 kB/s 
     |████████████████████████████████| 10.6 MB 6.2 MB/s 
     |████████████████████████████████| 2.9 MB 32.3 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 
     |████████████████████████████████| 1.4 MB 44.5 MB/s 
     |████████████████████████████████| 1.1 MB 62.7 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 1.3 MB 69.7 MB/s 
     |████████████████████████████████| 8.6 MB 28.7 MB/s 
     |████████████████████████████████| 2.2 MB 48.4 MB/s 
     |████████████████████████████████| 3.7 MB 49.2 MB/s 
     |████████████████████████████████| 184 kB 60.9 MB/s 
     |████████████████████████████████| 880 kB 70.2 MB/s 
  Created wheel for pygaggle: filename

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 15.7 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.0.0
    Uninstalling transformers-4.0.0:
      Successfully uninstalled transformers-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygaggle 0.0.3.1 requires tokenizers==0.9.4, but you have tokenizers 0.10.3 which is incompatible.
pygaggle 0.0.3.1 requires transformers==4.0.0, but you have transformers 4.6.1 which is incompatible.


In [6]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5, MonoBERT

class Reranker:

  def __init__(self):
    self.mono5t = MonoT5()
    self.monobert = MonoBERT()

  def rerank(self, query, texts, method="T5"):
    query = Query(query)
    texts = [Text(text, {"docid" : i}, 0) for i, text in enumerate(texts)]

    if method == "T5":
      reranker = self.mono5t
    if method == "BERT":
      reranker = self.monobert

    reranked = reranker.rerank(query, texts)
    reranked.sort(key=lambda x: x.score, reverse=True)

    return reranked

2022-09-12 12:10:05 [INFO] loader: Loading faiss with AVX2 support.
2022-09-12 12:10:05 [INFO] loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2022-09-12 12:10:05 [INFO] loader: Loading faiss.
2022-09-12 12:10:05 [INFO] loader: Successfully loaded faiss.


In [7]:
reranker = Reranker()

# QA Model

In [8]:
# Download zipped model
!gdown 10wA2fWuOUlGZCUDwUqruZSqDO5jn7HSg

# Unzip
!unzip 1892258.zip

# Delete
!rm 1892258.zip

Downloading...
From: https://drive.google.com/uc?id=10wA2fWuOUlGZCUDwUqruZSqDO5jn7HSg
To: /content/1892258.zip
100% 406M/406M [00:02<00:00, 136MB/s]
Archive:  1892258.zip
   creating: content/finbert-pretrain-finetuned-squad/model/
  inflating: content/finbert-pretrain-finetuned-squad/model/vocab.txt  
  inflating: content/finbert-pretrain-finetuned-squad/model/tokenizer.json  
  inflating: content/finbert-pretrain-finetuned-squad/model/pytorch_model.bin  
  inflating: content/finbert-pretrain-finetuned-squad/model/training_args.bin  
  inflating: content/finbert-pretrain-finetuned-squad/model/special_tokens_map.json  
  inflating: content/finbert-pretrain-finetuned-squad/model/config.json  
  inflating: content/finbert-pretrain-finetuned-squad/model/tokenizer_config.json  


In [9]:
# !pip install transformers 

from transformers import pipeline

In [10]:
model_dir = "content/finbert-pretrain-finetuned-squad/model"

question_answering = pipeline("question-answering", model=model_dir, tokenizer=model_dir)

In [11]:
# Example usage
context = """
Vía Célere intends to report on allocation of proceeds on
its website, on an annual basis, until full allocation. The allocation
reporting will include the total amount allocated to projects, the share
of financing vs. refinancing, and unallocated proceeds. In addition,
Vía Célere is committed to reporting on relevant impact metrics, such
as energy consumption reduction (in kWh) or emission reduction (in
tons of CO2e). Sustainalytics views Vía Célere’s allocation and impact
reporting as aligned with market practice.
"""

question = "how often is allocation of proceeds reported?"

In [12]:
result = question_answering(question=question, context=context, device=0)
print("Answer:", result['answer'])
print("Score:", result['score'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:316: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fw_args = {k: torch.tensor(v, device=self.device) for (k, v) in fw_args.items()}


Answer: annual
Score: 0.439126193523407


# Example Usage

In [13]:
# Download example PDF
!wget https://www.icmagroup.org/Emails/icma-vcards/V%C3%ADa%20C%C3%A9lere_External%20Review%20Report.pdf

--2022-09-12 12:11:35--  https://www.icmagroup.org/Emails/icma-vcards/V%C3%ADa%20C%C3%A9lere_External%20Review%20Report.pdf
Resolving www.icmagroup.org (www.icmagroup.org)... 91.216.93.249
Connecting to www.icmagroup.org (www.icmagroup.org)|91.216.93.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343324 (335K) [application/pdf]
Saving to: ‘Vía Célere_External Review Report.pdf.1’

Vía Célere_External 100%[===================>] 335.28K   946KB/s    in 0.4s    

2022-09-12 12:11:36 (946 KB/s) - ‘Vía Célere_External Review Report.pdf.1’ saved [343324/343324]



In [14]:
bm25_count = 50
reranked_count = 10

In [15]:
# Extract passages of text from PDF
filename = "Vía Célere_External Review Report.pdf"
reader = Reader(filename)
reader.extract_pdf()

In [16]:
queries = [
    "what did the use of proceeds finance?",
]

In [17]:
# Get BM25 rankings
try:
  texts = reader.get_ranked_texts(queries, n=bm25_count)
# No answers.
except IndexError:
  print("No Results in: {}".format(filename))

In [18]:
# Get T5 reranked passages
reranked = reranker.rerank(queries, texts, method="T5")
reranked = [item for item in reranked if len(item.text.strip())>0]
reranked = reranked[:reranked_count]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


In [26]:
for i, passage in enumerate(reranked):
  print("-"*80)
  print("Passage: {}".format(i))
  print("-"*15)
  print(passage.text)
  print("\n\n")
  if i == 3:
    break

--------------------------------------------------------------------------------
Passage: 0
---------------
 
Conclusion   
Vía Célere  has developed the Vía Célere Green Bo nd Framework  under which it will issue green bonds and the 
use of proceeds to finance energy -efficient residential assets.  Sustainalytics considers that the assets  funded 
by the green bond proceed s are expected to have  provide positive environmental impact .  
The Vía Célere Green Bond Framework  outlines a process by which proceeds will be tracked, allocated, and 
managed, and commitments have been made for reporting on the allocation and impact of th e use of 
proceeds. Furthermore, Sustainalytics believes that  the Vía Célere Green Bond Framework  is aligned with the 
overall sustainability strategy of the Company and that the green use of proceed s category will contribute to 
the advancement of  the UN Sus tainable Development Goa l 11. Additionally, Sustainalytics i s of the opinion 
that Vía Célere  

In [27]:
# Set context passage
context = reranked[0].text

# Set question.
question = queries[0]
# question = "ENTER QUESTION TEXT HERE"

In [28]:
# topk answers
topk = 3
results = question_answering(question=question, context=context, device=0, topk=topk)

In [30]:
for result in results:
  print(result,"\n")

{'score': 0.9140680432319641, 'start': 154, 'end': 190, 'answer': 'energy -efficient residential assets'} 

{'score': 0.04022157937288284, 'start': 172, 'end': 190, 'answer': 'residential assets'} 

{'score': 0.030510539188981056, 'start': 154, 'end': 191, 'answer': 'energy -efficient residential assets.'} 

